In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np

# Mediamill dataset

In [0]:
# data = pd.read_csv('/content/drive/My Drive/IPSC Final Project/Mediamill/mediamill_trSplit.txt', header = None)

In [0]:
# data1 = open('/content/drive/My Drive/IPSC Final Project/Mediamill/mediamill_trSplit.txt', 'r')
# data1.readlines()

# Bibtex dataset

In [0]:
# import scipy
# from scipy.io import arff
# data, meta = scipy.io.arff.loadarff('/content/drive/My Drive/IPSC Final Project/bibtex/bibtex-train.arff')
# df = pd.DataFrame(data)

# Randomly generated dataset

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
RandomForestClassifier(max_depth=2, random_state=0)
print(clf.feature_importances_)
# [0.14205973 0.76664038 0.0282433  0.06305659]
print(clf.predict([[0, 0, 0, 0]]))

[0.14205973 0.76664038 0.0282433  0.06305659]
[1]


# Yeast dataset

In [0]:
from scipy.io import arff
import pandas as pd

data = arff.loadarff('/content/drive/My Drive/IPSC Final Project/yeast/yeast-train.arff')
df = pd.DataFrame(data[0])
X = df.iloc[:,0:104]
y = df.iloc[: , 103:]
y = y.astype('int32')

# Train validation split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1200, 104)
(1200, 14)
(300, 104)
(300, 14)


# Random forest classifier

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, bootstrap=True)

# Accuracy of the classes

In [56]:
for category in y_test:
  # print(category)
  clf.fit(X_train, y_train[category])
  y_pred = clf.predict(X_test)
  # print(y_pred)
  # print(y_test[category])
  print(category, " : ", accuracy_score(y_test[category],y_pred))

Class1  :  1.0
Class2  :  0.7833333333333333
Class3  :  0.7566666666666667
Class4  :  0.76
Class5  :  0.77
Class6  :  0.7466666666666667
Class7  :  0.84
Class8  :  0.8366666666666667
Class9  :  0.95
Class10  :  0.8666666666666667
Class11  :  0.8366666666666667
Class12  :  0.7566666666666667
Class13  :  0.7433333333333333
Class14  :  0.9833333333333333
